# Train YOLOv4 for vehicle detection

Make sure to have GPU turned on in **EDIT > NOTEBOOK SETTING > NONE to GPU**

The YOLOv4 can be run using the following command
```bash
!./darknet detector test <path to .data file> <path to config> <path to weights> <path to image>
```

To show the image:
```bash
imShow('predictions.jpg')
```





In [ ]:
# This function display the image into console with given path 
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()


### Command line flags

**More commands in** https://github.com/AlexeyAB/darknet#how-to-use-on-the-command-line

Threshold Flag (Confidence level)
```
-thresh 0.2
```

the number 0.2 means any detected object above 20% percentage will be shown and consider as a result


Output Bounding Box Coordinate (output coord inside runtime code)

```Bash
-ext_output
```
You can output bounding box coordinates for each detection with the flag '-ext_output'. This external outputs flag will give you a few extra details about each detection within an image.


Don't Show Image

```Bash
-dont_show
```

The flag '-dont_show' will not output the image after running darknet. This doesn't really affect anything when running in Colab as the image is unable to output properly straight from darknet anyways. However, by adding the -dont_show flag you will get rid of the following warning from showing.

*This is an important flag to have when running darknet with YOLOv4 on video as it will suppress having the video shown.*

### Setup VM

In [ ]:
# clone AlexeyAB's darknet repo
!git clone https://github.com/AlexeyAB/darknet

# change makefile to enabled GPU and OPENCV 
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# make darknet (builds darknet so that we can run darknet executable file to run or train object detectors)
!make

# Download YOLOv4 pre-trained weight
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

### Mount Google drive

Images, dataset and configuration files can be imported or exported between google drive and Google Colab 


In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# create a symbolic link 
!ln -s /content/gdrive/My\ Drive/ /mydrive

# cd back into the darknet folder to run detections
%cd /content/darknet


**NOTE:** We are creating a symbolic link between '/content/gdrive/My\ Drive/' and '/mydrive.

It is a shortcut for '/mydrive' to map to the contents within the folder '/content/gdrive/My\ Drive/'.

The reason for this is that sometime having the space in 'My Drive' folder path can cause issues when running certain commands. This symbolic link will stop this from happening!



# **Setup files for training YOLOv4 Object Detector!**

In order to train the YOLOv4 detector we will need the following:

*   Labeled Custom Dataset
*   Custom .cfg file
*   obj.data and obj.names files
*   train.txt file (test.txt is optional here as well)

## Dataset

In [ ]:
# copy over both datasets into the root directory of the Colab VM (comment out test.zip if you are not using a validation dataset)
#!cp /mydrive/yolov4/obj.zip ../
#!cp /mydrive/yolov4/test.zip ../

# Import the datasets from the google drive into the root directory of Colab VM 
!cp /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Traffic images'/'wookr'/obj.zip ../
!cp /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Traffic images'/'wookr'/test.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
#!unzip ../obj.zip -d data/
#!unzip ../test.zip -d data/

# Unzip the dataset and put the content into /darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

## Config file

**Now edit the .cfg ,Open it up in a code or text editor to do so.**

*If you downloaded cfg to google drive you can use the built in  **Text Editor** by going to your google drive and double clicking on yolov4-obj.cfg and then clicking on the **Open with** drop down and selectin **Text Editor**.*

**How to Configure Your Variables:**

batch = 32

subdivisions = 16 **(If run into any issues then 32)**

width = 416

height = 416
**(these can be any multiple of 32, 416 is standard, you can sometimes improve results by making value larger like 608 but will slow down training)**

max_batches = (# of classes) * 2000
**(Mininum 6000 so if you are training for 1, 2, or 3 classes it will be 6000)**

steps = (80% of max_batches), (90% of max_batches)
**(so if your max_batches = 10000, then steps = 8000, 9000)**

filters = (# of classes + 5) * 3


*Optional: If you run into memory issues or find the training taking a super long time. In each of the three yolo layers in the cfg, change one line from random = 1 to random = 0 to speed up training but slightly reduce accuracy of model. Will also help save memory if you run into any memory issues.*


In [ ]:
# download or REPLACE cfg in google drive and change its name
#!cp cfg/yolov4-custom.cfg /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Colab Notebooks'/yolov4-obj.cfg

In [ ]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Colab Notebooks'/yolov4-obj.cfg ./cfg

## .data and .names files

### **obj.names** 

obj.names stores each class name per line in the **same order** as your classes.txt from the dataset generation step.

NOTE: You do not want to have spaces in your class name. Use _ to replace spaces

### **obj.data**

obj.data file is specifies where the path to the data. It includes: (change your number of classes accordingly, as well as your backup location)

**classes** = (# of class)

**train** = data/train.txt (train.txt path)

**valid** = data/test.txt (test.txt path / empty this whole line if no test data)

**names** = data/obj.names (obj.names path)

**backup** = /mydrive/backup (the backup folder path)


*This backup path is where we will save the weights to of our model throughout training. Create a backup folder in your google drive and put its correct path in this file.*

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Colab Notebooks'/obj.names ./data
!cp /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Colab Notebooks'/obj.data ./data

## train.txt

train.txt and test.txt hold the relative paths to all our training images and valdidation images.

We can use scripts from [theAIGuysCode's Github Repo](https://github.com/theAIGuysCode/YOLOv4-Cloud-Tutorial/tree/master/yolov4)

Download the two files to your local machine and upload them to your Google Drive so we can run them in the Colab Notebook.

In [ ]:
# Import the generate_train.py and generate_test.py script to cloud VM from Google Drive and run them

!cp /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Colab Notebooks'/generate_train.py ./
!cp /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Colab Notebooks'/generate_test.py ./

!python generate_train.py
!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

# Training YOLOv4

In [ ]:
# Download pre-trained weights for the convolutional layers. for faster training
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

Run the following command to train. (-dont_show flag stops chart from popping up since Colab Notebook can't open images on the spot, -map flag overlays mean average precision on chart to see how accuracy of your model is, only add map flag if you have a validation dataset)
```
!./darknet detector train <path to obj.data> <path to custom config> yolov4.conv.137 -dont_show -map
```
**TIP:** To avoid idle for too long and Google Colab closing the session, open up the inspector view (F12) on your browser.

Paste the following code into your console window and hit **Enter**
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```


In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture

# training starts from the pre-trained weight
# !./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

# kick off training from where it last saved (-map for validation and shows the graphing)
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/backup/yolov4-obj_1000.weights -dont_show -map -clear

In [ ]:
# show chart.png of how custom object detector did with training (available only if have validation dataset)
imShow('chart.png')
!cp chart.png /mydrive/backup/train_graph.png

Checking the Mean Average Precision (mAP) of Your Model
If you didn't run the training with the '-map- flag added then you can still find out the mAP of your model after training. Run the following command on any of the saved weights from the training to see the mAP value for that specific weight's file. I would suggest to run it on multiple of the saved weights to compare and find the weights with the highest mAP as that is the most accurate one!

NOTE: If you think your final weights file has overfitted then it is important to run these mAP commands to see if one of the previously saved weights is a more accurate model for your classes.

In [ ]:
# get the pre-trained weights from yolov4 ?
# cfg/coco.data cfg/yolov4.cfg yolov4.weights
# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/

In [ ]:
# Checking the mAP
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/backup/yolov4-obj_last.weights

# Done

In [ ]:
# set cfg to test mode 
%cd cfg
!sed -i 's/batch=32/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:
# set cfg to training mode
%cd cfg
!sed -i 's/batch=1/batch=32/' yolov4-obj.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' yolov4-obj.cfg
%cd ..

In [ ]:
# copy test images into VM
!cp -r /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Traffic images'/test_image ./data


In [ ]:
# change line 4 "test" to "test_image" and line 7 "data/test/" to "data/test_image"
!python generate_test.py

In [ ]:
# Test a folder of images
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/backup/yolov4-obj_1000.weights -thresh 0.2 -dont_show -ext_output < data/test.txt > result.txt

# Test a folder of videos (Not working)
# !./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/backup/yolov4-obj_1000.weights /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Traffic images'/test_extension_video/video_test.wmv -i 0 -dont_show -out_filename results1wmv.avi


In [ ]:
# wookr_yolov4-obj_1000.weights
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/backup/yolov4-obj_1000.weights /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Traffic images'/test_extension_image/test_extension/test9.tiff -thresh 0.2 -dont_show -ext_output > result.txt
# !./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/backup/yolov4-obj_1000.weights /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Traffic images'/test_extension_video/video_test.wmv -i 0 -dont_show -out_filename results1wmv.avi

In [ ]:
# run the custom detector 
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/backup/yolov4-obj_last.weights /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Traffic images'/test3.jpg -thresh 0.2 -dont_show -ext_output > result.txt

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Traffic images'/test3.jpg -thresh 0.2 -dont_show -ext_output > result.txt

In [ ]:
# GOOGLE DRIVE Export
# cp <copy file> <to path>
# !cp predictions.jpg /mydrive/detection1.jpg
!cp results1wmv.avi /mydrive/'FIT 3161 Vehicle Density Estimation For Traffic Prediction'/'Traffic images'/results1wmv.avi
# !cp result.txt /mydrive/result.txt